Using Tensor Flow and SciKit Learn to create a Neural Network in an attempt to create a predictive model:

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, roc_auc_score

# Load the diabetes dataset
engine = create_engine('sqlite:///(Full_Dataset)diabetesData.db')
diabetes_df = pd.read_sql_query('select * from diabetes', con=engine)

# Separate features (X) and target variable (y)
X = diabetes_df.drop("diabetes", axis=1)
y = diabetes_df["diabetes"].to_numpy()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Apply SMOTE to the training data
smote = SMOTE(random_state=1)
X_train_scaled_re, y_train_re = smote.fit_resample(X_train_scaled, y_train)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_dim = len(X_train_scaled_re[0])
node_value1 = 20
node_value2 = 10
node_value3 = 1
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=node_value1, activation="relu", input_dim=input_dim))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=node_value2, activation="relu",))

# Output layer
nn.add(tf.keras.layers.Dense(units=node_value3, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                440       
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 661
Trainable params: 661
Non-trainable params: 0
_________________________________________________________________


In [2]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [3]:
# Train the model
fit_model = nn.fit(X_train_scaled_re, y_train_re, epochs=50)

Epoch 1/50
10237/10237 [==============================] - 13s 1ms/step - loss: 0.4956 - accuracy: 0.7587
Epoch 2/50
10237/10237 [==============================] - 12s 1ms/step - loss: 0.4850 - accuracy: 0.7646
Epoch 3/50
10237/10237 [==============================] - 12s 1ms/step - loss: 0.4828 - accuracy: 0.7660
Epoch 4/50
10237/10237 [==============================] - 11s 1ms/step - loss: 0.4816 - accuracy: 0.7661
Epoch 5/50
10237/10237 [==============================] - 12s 1ms/step - loss: 0.4810 - accuracy: 0.7669
Epoch 6/50
10237/10237 [==============================] - 12s 1ms/step - loss: 0.4802 - accuracy: 0.7669
Epoch 7/50
10237/10237 [==============================] - 12s 1ms/step - loss: 0.4793 - accuracy: 0.7674
Epoch 8/50
10237/10237 [==============================] - 11s 1ms/step - loss: 0.4784 - accuracy: 0.7684
Epoch 9/50
10237/10237 [==============================] - 11s 1ms/step - loss: 0.4778 - accuracy: 0.7688
Epoch 10/50
10237/10237 [==============================

In [5]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Predict on the test set
predictions = nn.predict(X_test_scaled)

# Check the shape of y_test to handle both 1D and 2D cases
if y_test.ndim == 1:  # For binary classification where y_test is 1D
    auc_score = roc_auc_score(y_test, predictions)
    print(f"AUC Score: {auc_score}")

# Print additional classification metrics
print(classification_report(y_test, np.round(predictions)))

1982/1982 - 2s - loss: 0.4920 - accuracy: 0.7347 - 2s/epoch - 816us/step
Loss: 0.49199947714805603, Accuracy: 0.7346578240394592
1982/1982 [==============================] - 2s 753us/step
AUC Score: 0.81306226509885
              precision    recall  f1-score   support

         0.0       0.95      0.73      0.83     54551
         1.0       0.31      0.74      0.44      8869

    accuracy                           0.73     63420
   macro avg       0.63      0.74      0.63     63420
weighted avg       0.86      0.73      0.77     63420



Using Keras Tuner in an attempt to find the best predictive model:

In [6]:
import keras_tuner as kt

# Define the model-building function
def build_model(hp):
    model = tf.keras.models.Sequential()

    # First hidden layer: tuning the number of units
    model.add(tf.keras.layers.Dense(
        units=hp.Int('units_1', min_value=32, max_value=128, step=32),
        activation='relu',
        input_dim=X_train_scaled_re.shape[1]
    ))

    # Second hidden layer: tuning the number of units
    model.add(tf.keras.layers.Dense(
        units=hp.Int('units_2', min_value=16, max_value=64, step=16),
        activation='relu'
    ))

    # Output layer
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    # Compile the model: tuning the learning rate of Adam
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        ),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

# Initialize the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='accuracy',
    max_trials=10,
    executions_per_trial=3,
    directory='keras_tuner_logs',
    project_name='(Full_Dataset)keras_tuner_nn'
)

# Search for the best hyperparameters
tuner.search(X_train_scaled_re, y_train_re, epochs=10, validation_data=(X_test_scaled, y_test))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters: {best_hps.values}")

# Build and train the model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train_scaled_re, y_train_re, epochs=20, validation_data=(X_test_scaled, y_test))

Trial 10 Complete [00h 06m 41s]
accuracy: 0.7759474515914917

Best accuracy So Far: 0.7892017761866251
Total elapsed time: 01h 08m 10s
Best hyperparameters: {'units_1': 64, 'units_2': 64, 'learning_rate': 0.0008565404012436887}
Epoch 1/20
10237/10237 [==============================] - 15s 1ms/step - loss: 0.4871 - accuracy: 0.7634 - val_loss: 0.5181 - val_accuracy: 0.7172
Epoch 2/20
10237/10237 [==============================] - 14s 1ms/step - loss: 0.4760 - accuracy: 0.7699 - val_loss: 0.4991 - val_accuracy: 0.7322
Epoch 3/20
10237/10237 [==============================] - 13s 1ms/step - loss: 0.4697 - accuracy: 0.7736 - val_loss: 0.4785 - val_accuracy: 0.7445
Epoch 4/20
10237/10237 [==============================] - 14s 1ms/step - loss: 0.4642 - accuracy: 0.7775 - val_loss: 0.4960 - val_accuracy: 0.7371
Epoch 5/20
10237/10237 [==============================] - 14s 1ms/step - loss: 0.4596 - accuracy: 0.7802 - val_loss: 0.5143 - val_accuracy: 0.7215
Epoch 6/20
10237/10237 [=============

In [7]:
model_loss, model_accuracy = best_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Predict on the test set
predictions = best_model.predict(X_test_scaled)

# If it's a binary classification, you can calculate AUC and other metrics
# Check the shape of y_test to handle both 1D and 2D cases
if y_test.ndim == 1:  # For binary classification where y_test is 1D
    auc_score = roc_auc_score(y_test, predictions)
    print(f"AUC Score: {auc_score}")

# Print additional classification metrics
print(classification_report(y_test, np.round(predictions)))

1982/1982 - 2s - loss: 0.4932 - accuracy: 0.7472 - 2s/epoch - 901us/step
Loss: 0.4932153522968292, Accuracy: 0.7472406029701233
1982/1982 [==============================] - 2s 926us/step
AUC Score: 0.7928779373661036
              precision    recall  f1-score   support

         0.0       0.94      0.76      0.84     54551
         1.0       0.31      0.68      0.43      8869

    accuracy                           0.75     63420
   macro avg       0.62      0.72      0.63     63420
weighted avg       0.85      0.75      0.78     63420

